In [1]:
%%time

from IPython.display import clear_output

! pip install sentence_transformers==2.2.2

! pip install -qq -U langchain
! pip install -qq -U tiktoken
! pip install -qq -U pypdf
! pip install -qq -U faiss-gpu
! pip install -qq -U InstructorEmbedding

! pip install -qq -U transformers
! pip install -qq -U accelerate
! pip install -qq -U bitsandbytes

clear_output()

CPU times: user 313 ms, sys: 49.6 ms, total: 363 ms
Wall time: 33 s


In [2]:
%%time

import warnings
warnings.filterwarnings("ignore")

import os
import glob
import textwrap
import time

import langchain

### loaders
from langchain.document_loaders import PyPDFLoader, DirectoryLoader

### splits
from langchain.text_splitter import RecursiveCharacterTextSplitter

### prompts
from langchain import PromptTemplate, LLMChain

### vector stores
from langchain.vectorstores import FAISS

### models
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceInstructEmbeddings
### retrievers
from langchain.chains import RetrievalQA

import torch
import transformers
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline
)

clear_output()

CPU times: user 15.3 s, sys: 2.16 s, total: 17.4 s
Wall time: 34.5 s


In [3]:
!pip install -U langchain-community

In [4]:
print('langchain:', langchain.__version__)
print('torch:', torch.__version__)
print('transformers:', transformers.__version__)

langchain: 0.3.8
torch: 2.5.1+cu121
transformers: 4.46.3


In [5]:
import glob

# Specify the directory containing your data
files = sorted(glob.glob(r'C:\Users\DELL\Desktop\data\*'))

In [14]:
class CFG:
    # LLMs - Set the model name suitable for your project
    model_name = 'llama2-13b-chat'  # You can change to another model like 'mistral-7B' if needed
    temperature = 0  # Keep responses deterministic
    top_p = 0.95  # Use nucleus sampling for more diverse responses
    repetition_penalty = 1.15  # Penalty to prevent repetition

    # Splitting - Adjust chunk size based on the document's size
    split_chunk_size = 800  # Adjust based on the maximum input size of your model
    split_overlap = 0  # No overlap between chunks (you can change if needed)

    # Embeddings - Use a pre-trained sentence embedding model
    embeddings_model_repo = 'sentence-transformers/all-MiniLM-L6-v2'  # A lightweight embedding model

    # Similar passages - Retrieve the top k similar passages to improve context understanding
    k = 6

    # Paths - Update with your local directory paths
    PDFs_path = r'C:\Users\DELL\Desktop\data'  # Path to the folder containing Indian Constitution PDFs
    Embeddings_path = r'C:\Users\DELL\Desktop\data'  # Folder to store or load precomputed embeddings
    Output_folder = r'C:\Users\DELL\Desktop\data'  # Folder for the output vector database



In [73]:
from transformers import AutoTokenizer


model_repo = "daryl149/llama-2-7b-chat-hf"

print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)
print("Tokenizer loaded successfully.")


Loading tokenizer...
Tokenizer loaded successfully.


In [72]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from langchain.llms import HuggingFacePipeline

def get_model(model_choice):
    try:
        # Select a lightweight, publicly available model
        if model_choice == 'distilgpt2':
            model_repo = 'distilgpt2'  # Lightweight version of GPT-2
        else:
            raise ValueError("Model choice not recognized")

        # Load the tokenizer and model
        tokenizer = AutoTokenizer.from_pretrained(model_repo, use_fast=True)
        model = AutoModelForCausalLM.from_pretrained(model_repo)
        return tokenizer, model

    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None

# Select your model
model_choice = "distilgpt2"
tokenizer, model = get_model(model_choice)

if tokenizer is None or model is None:
    print("Error: Model or tokenizer could not be loaded.")
else:
    # Hugging Face pipeline for text generation
    pipe = pipeline(
        task="text-generation",
        model=model,
        tokenizer=tokenizer,
        pad_token_id=tokenizer.eos_token_id,
        max_length=1024,
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.2
    )

    # LangChain pipeline
    llm = HuggingFacePipeline(pipeline=pipe)

    print("Pipeline successfully set up.")


Pipeline successfully set up.


In [53]:
llm

HuggingFacePipeline(pipeline=<transformers.pipelines.text_generation.TextGenerationPipeline object at 0x7e905458f9d0>)

In [18]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 27.8 MB/s eta 0:00:00


In [21]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 11.8 MB/s eta 0:00:00


In [55]:
# Import necessary libraries
from google.colab import files
from langchain.document_loaders import PyMuPDFLoader

# Upload the PDF file from your local system
print("Please upload your PDF file:")
uploaded = files.upload()

# Retrieve the file name of the uploaded PDF
pdf_path = next(iter(uploaded.keys()))  # Get the first uploaded file

# Load the PDF using PyMuPDFLoader
loader = PyMuPDFLoader(pdf_path)
documents = loader.load()

# Display the number of pages loaded
print(f"Successfully loaded {len(documents)} pages from the PDF.")

# Display a preview of the loaded data (first 500 characters of the first document)
if documents:
    print("\nSample content from the first page:")
    print(documents[0].page_content[:500])  # Print first 500 characters


Please upload your PDF file:


Saving THE CONSTITUTION OF INDIA .pdf to THE CONSTITUTION OF INDIA  (3).pdf
Successfully loaded 60 pages from the PDF.

Sample content from the first page:
 
  1                             THE CONSTITUTION OF INDIA     THE CONSTITUTION OF INDIA  PREAMBLE  
WE, THE PEOPLE OF INDIA, having solemnly resolved to constitute India into a SOVEREIGN SOCIALIST 
SECULAR DEMOCRATIC REPUBLIC and to secure to all its citizens:  JUSTICE, social, economic and 
political;  LIBERTY of thought, expression, belief, faith and worship;  EQUALITY of status and of 
opportunity;  and to promote among them all  FRATERNITY assuring the dignity of the individual and the 
un


In [56]:
print(f'We have {len(documents)} pages in total')

We have 60 pages in total


In [57]:
documents[8].page_content

'clause (b) or clause (c) of article 39” (w.e.f. 3.1.1977). Section 4 has been declared invalid by  the \nSupreme Court in Minerva Mills Ltd. and others vs. Union of India and others  (1980) s.  2, S.C.C. 591. 2. \nIn Kesavananda Bharati vs. the State of Kerala (1973),Supp. S.C.R.1., the Supreme Court held the \nprovisions in italics to be invalid.  \n \n \n   15                             THE CONSTITUTION OF INDIA  (2) The Supreme Court shall have power to \nissue directions or orders or writs, including writs in the nature of habeas corpus, mandamus, \nprohibition, quo warranto and certiorari, whichever may be appropriate, for the enforcement of any of \nthe rights conferred by this Part. (3) Without prejudice to the powers conferred on the Supreme Court \nby clauses (1) and (2), Parliament may by law empower any other court to exercise within the local limits \nof its jurisdiction all or any of the powers exercisable by the Supreme Court under clause (2). (4) The \nright guaranteed

In [71]:

!pip install sentence-transformers huggingface_hub --upgrade

In [70]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from IPython.display import clear_output

# Define CFG
class CFG:
    split_chunk_size = 1000  # Size of each text chunk
    split_overlap = 200      # Overlap between chunks
    Output_folder = "output" # Define Output_folder within the CFG class

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CFG.split_chunk_size,
    chunk_overlap=CFG.split_overlap
)

# Split the loaded documents into chunks
texts = text_splitter.split_documents(documents)

# Output the result
print(f'We have created {len(texts)} chunks from {len(documents)} pages')

# Assuming necessary imports and setup for FAISS and HuggingFaceEmbeddings

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    texts,
    HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
)

### persist vector database
vectordb.save_local(f"{CFG.Output_folder}/faiss_index_rag") # save in output folder
#     vectordb.save_local(f"{CFG.Embeddings_path}/faiss_index_hp") # save in input folder

clear_output()

In [60]:
### test if vector DB was loaded correctly
vectordb.similarity_search('BERT')

[Document(metadata={'source': 'THE CONSTITUTION OF INDIA  (3).pdf', 'file_path': 'THE CONSTITUTION OF INDIA  (3).pdf', 'page': 40, 'total_pages': 60, 'format': 'PDF 1.5', 'title': '', 'author': 'pc', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'www.ilovepdf.com', 'creationDate': "D:20241118183538+00'00'", 'modDate': 'D:20241118183539Z', 'trapped': ''}, page_content='prorogue the House or either House; (b) dissolve the Legislative Assembly. 175. Right of Governor to \naddress and send messages to the House or Houses.—(1) The Governor may address the Legislative \nAssembly or, in the case of a State having a Legislative Council, either House of the Legislature of the \nState, or both Houses assembled together, and may for that purpose require the attendance of \nmembers. (2) The Governor may send messages to the House or Houses of the Legislature of the State, \nwhether with respect to a Bill then pending in the Legislature or otherwise, and a House to w

In [61]:
prompt_template = """
If you are unsure, do not fabricate an answer—simply state that you don't know.
Respond in the same language as the question was asked.
Base your answer solely on the following context provided below.

Context:
{context}

Question: {question}

Answer:
"""
from langchain.prompts import PromptTemplate

PROMPT = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)



**Retriever chain**

In [69]:
# Define CFG
class CFG:
    split_chunk_size = 1000  # Size of each text chunk
    split_overlap = 200      # Overlap between chunks
    Output_folder = "output" # Define Output_folder within the CFG class
    k = 4



retriever = vectordb.as_retriever(search_kwargs = {"k": CFG.k, "search_type" : "similarity"})

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff", # map_reduce, map_rerank, stuff, refine
    retriever = retriever,
    chain_type_kwargs = {"prompt": PROMPT},
    return_source_documents = True,
    verbose = False
)


In [63]:
### testing similarity search
question = "what is Article 14?"
vectordb.similarity_search(question, k = CFG.k)

[Document(metadata={'source': 'THE CONSTITUTION OF INDIA  (3).pdf', 'file_path': 'THE CONSTITUTION OF INDIA  (3).pdf', 'page': 33, 'total_pages': 60, 'format': 'PDF 1.5', 'title': '', 'author': 'pc', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word 2016', 'producer': 'www.ilovepdf.com', 'creationDate': "D:20241118183538+00'00'", 'modDate': 'D:20241118183539Z', 'trapped': ''}, page_content='five Judges and in the course of the hearing of the appeal the Court is satisfied that the appeal involves a \nsubstantial question of law as to the interpretation of this Constitution the determination of which is \nnecessary for the disposal of the appeal, such Court shall refer the question for opinion to a Court \nconstituted as required by this clause for the purpose of deciding any case involving such a question and \nshall on receipt of the opinion dispose of the appeal in conformity with such opinion. (4) No judgment \nshall be delivered by the Supreme Court save in open Court, and 

In [64]:
def wrap_text_preserve_newlines(text, width=700):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text


def process_llm_response(llm_response):
    ans = wrap_text_preserve_newlines(llm_response['result'])

    sources_used = ' \n'.join(
        [
            source.metadata['source'].split('/')[-1][:-4]
            + ' - page: '
            + str(source.metadata['page'])
            for source in llm_response['source_documents']
        ]
    )

    ans = ans + '\n\nSources: \n' + sources_used
    return ans

In [65]:
def llm_ans(query):
    start = time.time()

    llm_response = qa_chain.invoke(query)
    ans = process_llm_response(llm_response)

    end = time.time()

    time_elapsed = int(round(end - start, 0))
    time_elapsed_str = f'\n\nTime elapsed: {time_elapsed} s'
    return ans + time_elapsed_str

In [74]:
class CFG:
    split_chunk_size = 1000  # Size of each text chunk
    split_overlap = 200      # Overlap between chunks
    Output_folder = "output" # Define Output_folder within the CFG class
    k = 6                    # Number of neighbors for retrieval or other parameter
    model_name = "sentence-transformers/all-mpnet-base-v2"  # Specifying the model name for embeddings


In [75]:
query = "what is THE CONSTITUTION OF INDIA PREAMBLE?"
print(llm_ans(query))


If you are unsure, do not fabricate an answer—simply state that you don't know.
Respond in the same language as the question was asked.
Base your answer solely on the following context provided below.

Context:
1                             THE CONSTITUTION OF INDIA     THE CONSTITUTION OF INDIA  PREAMBLE
WE, THE PEOPLE OF INDIA, having solemnly resolved to constitute India into a SOVEREIGN SOCIALIST
SECULAR DEMOCRATIC REPUBLIC and to secure to all its citizens:  JUSTICE, social, economic and
political;  LIBERTY of thought, expression, belief, faith and worship;  EQUALITY of status and of
opportunity;  and to promote among them all  FRATERNITY assuring the dignity of the individual and the
unity and integrity of the Nation;  IN OUR CONSTITUENT ASSEMBLY this twenty-sixth day of November,
1949, do HEREBY ADOPT, ENACT AND GIVE TO OURSELVES THIS CONSTITUTION.


   2                             THE CONSTITUTION OF INDIA  PART I THE UNION AND ITS TERRITORY 1.
Name and territory of the Union